# Assignment CNN

In [1]:
# Pytorch functions
import torch
# Neural network layers
import torch.nn as nn
import torch.nn.functional as F
# Optimizer
import torch.optim as optim
# Handling dataset
import torch.utils.data as data
# Torchvision library
import torchvision
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
# For results
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import gc

import matplotlib.pyplot as plt
import numpy as np
import copy, time, os, cv2

In [2]:
SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
'''We define the transformations that will be applied when loading training and test set'''

train_transforms = transforms.Compose([
                   transforms.Resize((224,224)),
                   transforms.RandomHorizontalFlip(),
                   transforms.RandomPerspective(distortion_scale=0.5, p=0.3, fill=0),
                   transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(),]), p=0.3),
                   transforms.ToTensor(),
                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
                  transforms.Resize((224,224)),
                  transforms.Resize(256),
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

In [4]:
def plot_imgandlab():
  img = train_data.data[i] # Unnormalized images
  label = train_data[i][1]

In [5]:
ROOT = './data'

train_data = torchvision.datasets.CIFAR10(root=ROOT,
                                          train=True,
                                          download=True,
                                          transform=train_transforms)

test_data = torchvision.datasets.CIFAR10(root=ROOT,
                                         train=False,
                                         download=True,
                                         transform=test_transforms)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
class CNN(nn.Module):
  def __init__(self, layers):
    super().__init__()
    self.model = torchvision.models.resnet34(pretrained=True)
    self.model.fc = nn.Sequential(*layers)

    
  def forward(self, x):
    batch_size = x.shape[0]
    # Flatten x
    x = x.view(batch_size, -1)
    y = self.linear_layers(x)
    return y
  
  def count_parameters(self, info=False):
    nparams = sum(p.numel() for p in self.parameters() if p.requires_grad)
    if info:
      print("The model has",nparams,"trainable parameters.")
    return nparams

In [7]:
def create_validation_set(train_data, train_percentage, test_transforms):
  num_train_examples = int(len(train_data) * train_percentage)
  num_valid_examples = len(train_data) - num_train_examples

  train_data, valid_data = data.random_split(train_data, [num_train_examples, num_valid_examples])

  print(f"Number training examples: {len(train_data)}")
  print(f"Number validation examples: {len(valid_data)}")

  valid_data = copy.deepcopy(valid_data)
  valid_data.dataset.transform = test_transforms

  return valid_data

In [8]:
valid_data=create_validation_set(train_data,0.8,test_transforms)

Number training examples: 40000
Number validation examples: 10000


In [9]:
INPUT_DIM = 224*224*3
OUTPUT_DIM = 10 # 10 classes

layers = [nn.Linear(in_features=512,out_features=512,bias=True),
          nn.ReLU(),
          nn.BatchNorm1d(512,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True),
          nn.Dropout(p=0.3),
          nn.Linear(in_features=512,out_features=300,bias=True),
          nn.ReLU(),
          nn.BatchNorm1d(300,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True),
          nn.Dropout(p=0.2),
          nn.Linear(in_features=300,out_features=OUTPUT_DIM)
]
#layers = [nn.Linear(in_features=512,out_features=512,bias=True),nn.Dropout(p=0.2),nn.Linear(in_features=512,out_features=OUTPUT_DIM)]

myCNN = CNN(layers)
myCNN.count_parameters(info=True)

# Create iterators
BATCH_SIZE = 256

train_iterator = torch.utils.data.DataLoader(train_data, 
                                             shuffle=True, 
                                             batch_size=BATCH_SIZE)

valid_iterator = torch.utils.data.DataLoader(valid_data, 
                                             batch_size=BATCH_SIZE)

test_iterator = torch.utils.data.DataLoader(test_data, 
                                            batch_size=BATCH_SIZE)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth



The model has 21705862 trainable parameters.


In [10]:
# Loss
criterion = nn.CrossEntropyLoss() # Softmax + CrossEntropy

# Put model&criterion on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = criterion.to(device)

# Optim
optimizer=optim.Adam(myCNN.parameters(),lr=.5e-4, weight_decay=.8e-5)

In [11]:
def train(model, iterator, optimizer, criterion, device):
  epoch_loss = 0
  epoch_acc = 0

  # Train mode
  model.train()

  for (x,y) in iterator:
    x = x.to(device)
    y = y.to(device)
    # Set gradients to zero
    optimizer.zero_grad()
    
    # Make Predictions
    y_pred = model(x)

    # Compute loss
    loss = criterion(y_pred, y)
    
    # Compute accuracy
    acc = calculate_accuracy(y_pred, y)

    # Backprop
    loss.backward()

    # Apply optimizer
    optimizer.step()

    # Extract data from loss and accuracy
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss/len(iterator), epoch_acc/len(iterator)

def evaluate(model, iterator, criterion, device):
  epoch_loss = 0
  epoch_acc = 0

  # Evaluation mode
  model.eval()

  # Do not compute gradients
  with torch.no_grad():

    for (x,y) in iterator:

      x = x.to(device)
      y = y.to(device)
      
      # Make Predictions
      y_pred = model(x)

      # Compute loss
      loss = criterion(y_pred, y)
      
      # Compute accuracy
      acc = calculate_accuracy(y_pred, y)

      # Extract data from loss and accuracy
      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss/len(iterator), epoch_acc/len(iterator)

def calculate_accuracy(y_pred, y):
  '''
  Compute accuracy from ground-truth and predicted labels.
  
  Input
  ------
  y_pred: torch.Tensor [BATCH_SIZE, N_LABELS]
  y: torch.Tensor [BATCH_SIZE]

  Output
  ------
  acc: float
    Accuracy
  '''
  y_prob = F.softmax(y_pred, dim = -1)
  y_pred = y_pred.argmax(dim=1, keepdim = True)
  correct = y_pred.eq(y.view_as(y_pred)).sum()
  acc = correct.float()/y.shape[0]
  return acc

def model_training(n_epochs, model, train_iterator, valid_iterator, optimizer, criterion, device, model_name='best_model.pt'):

  # Initialize validation loss
  best_valid_loss = float('inf')

  # Save output losses, accs
  train_losses = []
  train_accs = []
  valid_losses = []
  valid_accs = []

  # Loop over epochs
  for epoch in range(n_epochs):
    start_time = time.time()
    # Train
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    # Validation
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    # Save best model
    if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      # Save model
      torch.save(model.state_dict(), model_name)
    end_time = time.time()
    
    print(f"\nEpoch: {epoch+1}/{n_epochs} -- Epoch Time: {end_time-start_time:.2f} s")
    print("---------------------------------")
    print(f"Train -- Loss: {train_loss:.3f}, Acc: {train_acc * 100:.2f}%")
    print(f"Val -- Loss: {valid_loss:.3f}, Acc: {valid_acc * 100:.2f}%")

    # Save
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)

  return train_losses, train_accs, valid_losses, valid_accs

In [12]:
torch.cuda.is_available()

True

In [ ]:
N_EPOCHS = 12
train_losses, train_accs, valid_losses, valid_accs = model_training(N_EPOCHS, 
                                                                    myCNN.model.to(device), 
                                                                    train_iterator, 
                                                                    valid_iterator, 
                                                                    optimizer, 
                                                                    criterion, 
                                                                    device,
                                                                    'mlp.pt')


Epoch: 1/12 -- Epoch Time: 1186.11 s
---------------------------------
Train -- Loss: 0.581, Acc: 82.81%
Val -- Loss: 0.202, Acc: 94.88%

Epoch: 2/12 -- Epoch Time: 1190.16 s
---------------------------------
Train -- Loss: 0.167, Acc: 94.97%
Val -- Loss: 0.122, Acc: 96.75%

Epoch: 3/12 -- Epoch Time: 1191.32 s
---------------------------------
Train -- Loss: 0.111, Acc: 96.65%
Val -- Loss: 0.088, Acc: 97.58%

Epoch: 4/12 -- Epoch Time: 1192.39 s
---------------------------------
Train -- Loss: 0.081, Acc: 97.50%
Val -- Loss: 0.065, Acc: 98.03%

Epoch: 5/12 -- Epoch Time: 1192.16 s
---------------------------------
Train -- Loss: 0.065, Acc: 98.01%
Val -- Loss: 0.048, Acc: 98.80%

Epoch: 6/12 -- Epoch Time: 1192.51 s
---------------------------------
Train -- Loss: 0.051, Acc: 98.48%
Val -- Loss: 0.066, Acc: 97.93%

Epoch: 7/12 -- Epoch Time: 1191.59 s
---------------------------------
Train -- Loss: 0.043, Acc: 98.71%
Val -- Loss: 0.035, Acc: 99.02%

Epoch: 8/12 -- Epoch Time: 1192.7

In [ ]:
def plot_results(n_epochs, train_losses, train_accs, valid_losses, valid_accs):
  N_EPOCHS = n_epochs
  # Plot results
  plt.figure(figsize=(20, 6))
  _ = plt.subplot(1,2,1)
  plt.plot(np.arange(N_EPOCHS)+1, train_losses, linewidth=3)
  plt.plot(np.arange(N_EPOCHS)+1, valid_losses, linewidth=3)
  _ = plt.legend(['Train', 'Validation'])
  plt.grid('on'), plt.xlabel('Epoch'), plt.ylabel('Loss')

  _ = plt.subplot(1,2,2)
  plt.plot(np.arange(N_EPOCHS)+1, train_accs, linewidth=3)
  plt.plot(np.arange(N_EPOCHS)+1, valid_accs, linewidth=3)
  _ = plt.legend(['Train', 'Validation'])
  plt.grid('on'), plt.xlabel('Epoch'), plt.ylabel('Accuracy')

In [ ]:
plot_results(N_EPOCHS,train_losses,train_accs,valid_losses,valid_accs)

In [ ]:
def model_testing(model, test_iterator, criterion, device, model_name='mlp.pt'):
  # Test model
  model.load_state_dict(torch.load(model_name))
  test_loss, test_acc = evaluate(model, test_iterator, criterion, device)
  print(f"Test -- Loss: {test_loss:.3f}, Acc: {test_acc * 100:.2f} %")

In [ ]:
model_testing(myCNN.model.to(device),test_iterator,criterion,device)

In [ ]:
def predict(model, iterator, device):
  
  # Evaluation mode
  model.eval()
  
  labels = []
  pred = []

  with torch.no_grad():
    for (x, y) in iterator:
      x = x.to(device)
      y_pred = model(x)

      # Get label with highest score
      y_prob = F.softmax(y_pred, dim = -1)
      top_pred = y_prob.argmax(1, keepdim=True)

      labels.append(y.cpu())
      pred.append(top_pred.cpu())

  labels = torch.cat(labels, dim=0)
  pred = torch.cat(pred, dim=0)
  
  return labels, pred

def print_report(model, test_iterator, device):
  labels, pred = predict(model, test_iterator, device)
  print(confusion_matrix(labels, pred))
  print("\n")
  print(classification_report(labels, pred))

In [ ]:
print_report(myCNN.model.to(device),test_iterator,device)